 # Generating  and Visualizing Charging Stations


### Installing and importing modules

In [1]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
#!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
!pip install geopandas 
import geopandas

     |████████████████████████████████| 972kB 2.8MB/s 
     |████████████████████████████████| 10.9MB 14.4MB/s 
     |████████████████████████████████| 14.8MB 323kB/s 


In [4]:
!pip install google-search-results #Installing the Serp Google search API

In [5]:
from serpapi.google_search_results import GoogleSearchResults #Importing the GoogleSearchResults function

## Data Collection and Preprocessing

Reading a Geojson file that contains the list of all countries in the world

In [6]:
geo_df = geopandas.read_file('https://datahub.io/core/geo-countries/r/countries.geojson')

In [7]:
geo_df.head() #Viewing the contents

,ADMIN,ISO_A3,geometry
0,Aruba,ABW,"POLYGON ((-69.99694 12.57758, -69.93639 12.531..."
1,Afghanistan,AFG,"POLYGON ((71.04980 38.40866, 71.05714 38.40903..."
2,Angola,AGO,"MULTIPOLYGON (((11.73752 -16.69258, 11.73851 -..."
3,Anguilla,AIA,"MULTIPOLYGON (((-63.03767 18.21296, -63.09952 ..."
4,Albania,ALB,"POLYGON ((19.74777 42.57890, 19.74601 42.57993..."


Next up, we extract european data from this dataframe.

In [8]:
eu = ['Albania', 'Austria', 'Belgium', 'Bosnia and Herzegovina', 'Bulgaria', 'Croatia', 'Cyprus',
       'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France','Russia',
       'Germany', 'Hungary', 'Italy','Kosovo', 'Latvia',
       'Lithuania', 'Luxembourg', 'Malta', 'Moldova', 'Monaco','Montenegro', 'Netherlands', 'Macedonia','Norway', 'Poland', 
        'Romania', 'San Marino', 'Republic of Serbia','Switzerland', 'Slovakia', 'Slovenia',
       'Sweden', 'Thessaloniki', 'United Kingdom']
len(eu)
x = [i for i in geo_df.index
 if geo_df.loc[i,'ADMIN'] in eu]
eu_df = geo_df.loc[x,:]
eu_df[['ADMIN','ISO_A3']].head()

,ADMIN,ISO_A3
4,Albania,ALB
16,Austria,AUT
19,Belgium,BEL
23,Bulgaria,BGR
26,Bosnia and Herzegovina,BIH


Here, we use the Geocode library to obtain the geographical coordinates of the
european countries.

In [9]:
coordinates = {'country':[],'latitude':[],'longitude':[]}
for i in eu_df['ADMIN']:
    address = i
    geolocator = Nominatim(user_agent="eu_explorer")
    location = geolocator.geocode(address)
    try:
        coordinates['latitude'].append(location.latitude)
        coordinates['longitude'].append(location.longitude)
        coordinates['country'].append(i)
    except AttributeError as e:
        print(f'{i}  \n{e}')

In [ ]:
europe_df = pd.DataFrame(coordinates)
europe_df.head()

,country,latitude,longitude
0,Albania,41.000028,19.999962
1,Austria,47.200034,13.199959
2,Belgium,50.640281,4.666715
3,Bulgaria,42.607397,25.485662
4,Bosnia and Herzegovina,44.305348,17.596147


We then store the coordinates of london as variables for later use.

In [ ]:
address = 'London'

geolocator = Nominatim(user_agent="lag_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of {} are {}, {}.'.format(address,latitude, longitude))

The geograpical coordinates of London are 51.5073219, -0.1276474.


A dataframe is initialized with defined columnns but no data...yet.

In [ ]:
stations_df = pd.DataFrame(columns=['title', 'address', 'lat', 'lng', 'type','place_id_search'])
stations_df

,title,address,lat,lng,type,place_id_search


The function that uses the Serp API to extract the charging stations based on a query and some coordinates with the aid of an API key is created below.

In [ ]:
def get_chargers(query, latitude, longitude):
  params = {
        "engine": "google_maps",
        "q": query,
        "google_domain": "google.com",
        'api_key': 'Enter API Key',
        "type": "search",
        "ll": f"@{latitude},{longitude},12z",
        'output':'json'
    }

  client = GoogleSearchResults(params)
  data = client.get_dict()
  df = pd.DataFrame(data['local_results'])
  df['lat']= [x['latitude'] for x in df['gps_coordinates']]
  df['lng']= [x['longitude'] for x in df['gps_coordinates']]
  
  return df[['title', 'address', 'lat', 'lng', 'type','place_id_search']]

The function above is called and the results stored in the pre-defined dataframe.

In [ ]:
queries = ['charger','charging station','chargepoint','charge']

for query in queries:
  for latitude, longitude in europe_df[['latitude','longitude']].values:
    try:
      inter = get_chargers(query, latitude, longitude)
      print(stations_df.shape)
      stations_df = stations_df.append(inter, ignore_index=True)
    except KeyError:
      print("Empty query.")
      print(query,latitude,longitude)
    

(0, 6)
(20, 6)
(40, 6)
(60, 6)
(80, 6)
(100, 6)
(120, 6)
(140, 6)
(154, 6)
(166, 6)
(177, 6)
(185, 6)
(205, 6)
(225, 6)
(245, 6)
(265, 6)
(285, 6)
(305, 6)
(325, 6)
(327, 6)
(347, 6)
(367, 6)
(387, 6)
(407, 6)
(427, 6)
(447, 6)
(467, 6)
(485, 6)
(505, 6)
Empty query.
charger 45.9852129 24.6859225
(525, 6)
(545, 6)
(565, 6)
(585, 6)
(593, 6)
(613, 6)
(633, 6)
(642, 6)
(662, 6)
(682, 6)
(702, 6)
(713, 6)
(733, 6)
(753, 6)
(773, 6)
(793, 6)
(813, 6)
(833, 6)
(853, 6)
(873, 6)
(893, 6)
(913, 6)
(933, 6)
(953, 6)
(970, 6)
(981, 6)
(1001, 6)
(1021, 6)
(1041, 6)
(1047, 6)
(1067, 6)
(1087, 6)
(1103, 6)
(1123, 6)
(1143, 6)
(1163, 6)
(1182, 6)
(1202, 6)
(1222, 6)
(1242, 6)
(1262, 6)
(1282, 6)
(1302, 6)
(1322, 6)
(1342, 6)
(1362, 6)
(1382, 6)
(1402, 6)
(1422, 6)
(1442, 6)
(1462, 6)
(1482, 6)
(1502, 6)
(1522, 6)
(1542, 6)
(1562, 6)
(1582, 6)
(1593, 6)
(1613, 6)
(1625, 6)
(1645, 6)
(1665, 6)
(1676, 6)
(1696, 6)
(1716, 6)
(1736, 6)
(1756, 6)
(1776, 6)
(1796, 6)
(1816, 6)
(1836, 6)
(1856, 6)
(1876, 6

In [9]:
print(stations_df.shape)
stations_df.head()

The following coordinates represent regions on the map found to sparsely contain charging stations.

In [10]:
exclusions = [(49.6321, 2.5488),(51.1725, 0.1758), ( 51.1346,-0.3680), (51.1966,1.0162),
 ( 50.7782, 2.3016), (51.5053, 4.9274), (51.1311, 4.2023),( 50.8718, 3.4882),
 (51.3443, 5.9106), (51.3718, 7.5366), (50.5274, 9.0308),  (49.1673, 3.2959),
 (52.7097, 12.9419), (52.8824, 7.7344), (51.6044, 13.9746), ( 49.0667, 10.4150),
(51.9984, 16.3477), (53.6707, 17.6660), (53.0940, 15.9082), ( 50.5693, 18.7646),
 ( 51.2894, 21.9727), (50.3455, 21.3135), (54.1367, 21.1377), (53.1468, 22.2803),
 (54.0336, 9.7119),( 54.5466, 9.3604),(55.1977, 9.1846),(55.2040, 10.4590),(54.9587, 9.9042),(54.8418
, 9.1461),(54.1657, 10.5304),(54.2941, 10.9644),(54.4605, 11.1566),(54.8070
, 11.2830),( 54.7310, 11.6071),(54.7975, 11.9586),(54.6643, 11.9312),(53.9496
, 12.2827),(53.6577, 11.7224),(53.6642, 12.6013),( 53.3571, 12.9529),( 52.9817
, 13.2605),(53.3309,27.7734),(52.8028, 25.3564),( 55.0028, 28.6523),( 52.4828
, 29.6631),(48.9514, 24.6973),( 51.0966, 25.8398),(49.4110, 27.6855),(50.3735
, 30.4980),(47.9899, 31.2891),(47.6654, 34.5850),( 47.7245, 37.2656),(48.9225
, 37.8369),(48.7779, 36.1670),(49.2965, 34.7607),(51.1793, 32.6514),(50.5134
, 33.7500),(58.0779, 28.7183),( 57.1482, 29.5093),(56.3653, 29.9268),( 58.0314
, 26.8066),(59.1421, 26.4771),(59.1534, 29.0259),(59.3556, 30.5859),( 59.7343
, 32.3657),( 58.6427, 33.5083),(58.0546, 32.3657),(57.5983, 31.3989),(58.3441
, 30.7178),(58.0197, 36.4966),(57.2434, 36.6504),(57.3858, 34.9365),( 56.6683
, 34.4751),(56.2433, 32.6733),(56.6321, 31.9702),(56.6562, 31.5308),(57.2790
, 34.3433),(55.8136, 36.2769),(56.1455, 37.6831),(55.2165, 38.3643),(53.9432
, 37.5732),(54.6992, 35.9473),(54.1367, 35.0024),(55.2416, 34.1016),(54.3678
, 32.4316),( 55.1789, 31.9043),(52.8293, 36.3867),(51.6317, 35.5298),(51.6180
, 36.9800),(52.6964, 34.0796),(52.6564, 32.4976),(53.3309, 33.5083),(51.6453
, 35.9473),(50.7086, 37.5732),(54.1367, 39.5068),(52.6164, 39.0234),(52.5630
, 41.5723),( 52.5363, 41.6162),(51.3718, 39.5947),(50.5414, 40.0781)]

The function that calls the serpapi is called again with these excluded coordinates as the focus.

In [ ]:
queries = ['charger','charging station','chargepoint','charge']#supercharger

for query in queries:
  print(query)
  for lat, lng in exclusions:
    try:
      inter = get_chargers(query,lat,lng)
      stations_df = stations_df.append(inter,ignore_index=True)
    except:
      print(f'I faced an error with the query {query} around {lat}, {lng}')
stations_df.shape

charger
I faced an error with the query charger around 53.1468, 22.2803
I faced an error with the query charger around 48.7779, 36.167
I faced an error with the query charger around 57.1482, 29.5093
I faced an error with the query charger around 50.7086, 37.5732
charging station
I faced an error with the query charging station around 53.6642, 12.6013
I faced an error with the query charging station around 48.9225, 37.8369
chargepoint
I faced an error with the query chargepoint around 53.6642, 12.6013
I faced an error with the query chargepoint around 57.3858, 34.9365
I faced an error with the query chargepoint around 55.8136, 36.2769
I faced an error with the query chargepoint around 52.6964, 34.0796
charge
I faced an error with the query charge around 53.1468, 22.2803
I faced an error with the query charge around 58.6427, 33.5083
I faced an error with the query charge around 55.2416, 34.1016
I faced an error with the query charge around 54.3678, 32.4316


(9357, 6)

The data collected into stations_df is filtered to make sure we have just chargng stations.

In [ ]:
cond= stations_df['type'] == 'Electric vehicle charging station'
ind = stations_df.index[cond]
stations_df = stations_df.iloc[ind,:]
stations_df = stations_df.drop_duplicates()

### Extracting Belarus Data

The relevant libraries are imported and a request is made to [this webpage](https://www.belorusneft.by/beloil-map/?lang=en#)

In [16]:
from bs4 import BeautifulSoup
import requests
import re

r = requests.get("https://www.belorusneft.by/beloil-map/?lang=en#")

In [12]:
trs = BeautifulSoup(r.text)

The important sections on the source code of the website is obtained.

In [13]:
trs = """
<tbody>
   	<tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(54.154996, 27.834638, 33);" title="Show on map"></div></td><td class="sorting_1">01</td><td></td><td></td><td></td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.931734, 27.511584, 50);" title="Show on map"></div></td><td class="sorting_1">01</td><td></td><td>Minsk, Pobediteley Avenue, 20</td><td>+375(17)279-88-64</td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.846632, 27.655503, 47);" title="Show on map"></div></td><td class="sorting_1">010</td><td></td><td>Minsk, Svislochskaya street, 11</td><td>+375(17)279-88-64</td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.899066, 27.565566, 48);" title="Show on map"></div></td><td class="sorting_1">011</td><td></td><td>Minsk, Kirova street, 18</td><td>+375(17)279-88-64</td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.944958, 27.599032, 44);" title="Show on map"></div></td><td class="sorting_1">012</td><td></td><td>Minsk, Lukyanovicha street, 10</td><td>+375(17)279-88-64</td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.90808, 27.49465, 43);" title="Show on map"></div></td><td class="sorting_1">013</td><td></td><td>Minsk, Pushkin Avenue, 39</td><td>+375(17)279-88-64</td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.912085, 27.53795, 35);" title="Show on map"></div></td><td class="sorting_1">014</td><td></td><td>Minsk, Pobediteley Avenue, 31-1</td><td>+375(17)279-88-64</td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.911331, 27.548964, 16);" title="Show on map"></div></td><td class="sorting_1">015</td><td></td><td>Minsk, Pobediteley Avenue, 4</td><td>+375(17)279-88-64</td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.923681, 27.517642, 14);" title="Show on map"></div></td><td class="sorting_1">016</td><td></td><td>Minsk, Pobediteley Avenue, 63</td><td>+375(17)279-88-64</td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.934779, 27.48458, 10);" title="Show on map"></div></td><td class="sorting_1">017</td><td></td><td>Minsk, Pobediteley Avenue, 111</td><td>+375(17)279-88-64</td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.953373, 27.451354, 13);" title="Show on map"></div></td><td class="sorting_1">018</td><td></td><td>Minsk, Pobediteley Avenue, 120</td><td>+375(17)279-88-64</td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.954088, 27.712964, 54);" title="Show on map"></div></td><td class="sorting_1">019</td><td></td><td>Minsk city, Nezavisimosti Avenue, 193</td><td>+375(17)279-88-64</td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.903902, 27.977725, 34);" title="Show on map"></div></td><td class="sorting_1">02</td><td></td><td></td><td></td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.937676, 27.491276, 38);" title="Show on map"></div></td><td class="sorting_1">02</td><td></td><td>Minsk, Pobediteley Avenue, 84</td><td>+375(17)279-88-64</td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.954655, 27.714536, 53);" title="Show on map"></div></td><td class="sorting_1">020</td><td></td><td>Minsk city, Nezavisimosti Avenue, 195</td><td>+375(17)279-88-64</td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.950416, 27.70679, 55);" title="Show on map"></div></td><td class="sorting_1">021</td><td></td><td>Minsk city, Nezavisimosti Avenue, 196</td><td>+375(17)279-88-64</td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.864503, 27.482044, 12);" title="Show on map"></div></td><td class="sorting_1">022</td><td></td><td>Minsk city, Gazeta Pravda Avenue, 25</td><td>+375(17)279-88-64</td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.891153, 27.552995, 52);" title="Show on map"></div></td><td class="sorting_1">023</td><td></td><td>Minsk city, Bobruiskaya street, 4</td><td>+375(17)279-88-64</td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.891172, 27.552911, 9);" title="Show on map"></div></td><td class="sorting_1">024</td><td></td><td>Minsk city, Bobruiskaya street, 4а</td><td>+375(17)279-88-64</td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.841543, 27.628851, 51);" title="Show on map"></div></td><td class="sorting_1">025</td><td></td><td>Minsk city, Tashkentskaya street, 19</td><td>+375(17)279-88-64</td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.930486, 27.579231, 56);" title="Show on map"></div></td><td class="sorting_1">026</td><td></td><td>Minsk city, Surganova street, 4</td><td>+375(17)279-88-64</td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.890385, 28.032155, 42);" title="Show on map"></div></td><td class="sorting_1">027</td><td></td><td>Minsk city, the territory of the National Airport "Minsk"</td><td>+375(17)279-88-64</td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.922331, 27.526299, 17);" title="Show on map"></div></td><td class="sorting_1">028</td><td></td><td>Minsk city, Pobediteley avenue, 59</td><td>+375(17)279-88-64</td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.911447, 27.544808, 8);" title="Show on map"></div></td><td class="sorting_1">029</td><td></td><td>Minsk city, Pobediteley avenue, 19</td><td>+375(17)279-88-64</td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.860084, 27.478316, 49);" title="Show on map"></div></td><td class="sorting_1">03</td><td></td><td>Minsk, Dzerzhinsky Avenue, 104</td><td>+375(17)279-88-64</td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.896057, 27.556896, 15);" title="Show on map"></div></td><td class="sorting_1">030</td><td></td><td>Minsk city, Kirova street, 15</td><td>+375(17)279-88-64</td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.878398, 27.533519, 11);" title="Show on map"></div></td><td class="sorting_1">031</td><td></td><td>Minsk city, Brilevskaya street, 2</td><td>+375(17)279-88-64</td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.897044, 27.550231, 6);" title="Show on map"></div></td><td class="sorting_1">032</td><td></td><td>Minsk city, Nezavisimosti Avenue, 11</td><td>+375(17)279-88-64</td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.893896, 27.550029, 7);" title="Show on map"></div></td><td class="sorting_1">033</td><td></td><td>Minsk city, Nezavisimosti Avenue, 8</td><td>+375(17)279-88-64</td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.89709, 27.555977, 22);" title="Show on map"></div></td><td class="sorting_1">034</td><td></td><td>Minsk city, Karla Marksa street, 14</td><td>+375(17)279-88-64</td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.930608, 27.64704, 37);" title="Show on map"></div></td><td class="sorting_1">035</td><td></td><td>Minsk city, Zhasminovaya street</td><td>+375(17)279-88-64</td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.927489, 27.437317, 88);" title="Show on map"></div></td><td class="sorting_1">036</td><td></td><td>Minsk city, Nalibokskaia street, 10</td><td>+375(29)643-14-31</td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.94414, 27.665928, 63);" title="Show on map"></div></td><td class="sorting_1">037</td><td></td><td>Minsk city, Nezavisimosti Avenue, 169</td><td>+375(29)643-14-31</td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.914261, 27.415612, 90);" title="Show on map"></div></td><td class="sorting_1">038</td><td></td><td>Minsk city, Kamenogorskaia street, 28</td><td>+375(29)643-14-31</td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.860428, 27.573761, 64);" title="Show on map"></div></td><td class="sorting_1">039</td><td></td><td>Minsk city, Mayakovskogo street, 10</td><td>+375(29)643-14-31</td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.909889, 27.545799, 46);" title="Show on map"></div></td><td class="sorting_1">04</td><td></td><td>Minsk, Pobediteley Avenue, 9</td><td>+375(17)279-88-64</td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.897119, 27.546309, 65);" title="Show on map"></div></td><td class="sorting_1">040</td><td></td><td>Minsk city, Bersona street, 14</td><td>+375(29)643-14-31</td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.91876, 27.603535, 91);" title="Show on map"></div></td><td class="sorting_1">041</td><td></td><td>Minsk city, Surganova street, 2</td><td>+375(29)643-14-31</td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.886124, 27.581914, 66);" title="Show on map"></div></td><td class="sorting_1">042</td><td></td><td>Minsk city, Partizanskogo Avenue, 2</td><td>+375(29)643-14-31</td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.90629, 27.492553, 67);" title="Show on map"></div></td><td class="sorting_1">043</td><td></td><td>Minsk city, Dunina-Marcinkevicha street, 8</td><td>+375(29)643-14-31</td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.91583, 27.541385, 68);" title="Show on map"></div></td><td class="sorting_1">044</td><td></td><td></td><td>+375(29)643-14-31</td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.901167, 27.56964, 69);" title="Show on map"></div></td><td class="sorting_1">045</td><td></td><td>Minsk city, Yanki Kupaly street,7</td><td>+375(29)643-14-31</td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.860213, 27.607281, 70);" title="Show on map"></div></td><td class="sorting_1">046</td><td></td><td>Minsk city, Rokossovskogo Avenue, 85</td><td>+375(29)643-14-31</td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.930918, 27.54819, 71);" title="Show on map"></div></td><td class="sorting_1">047</td><td></td><td>Minsk city, Chervyakova street, 64</td><td>+375(29)643-14-31</td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.848452, 27.460501, 72);" title="Show on map"></div></td><td class="sorting_1">048</td><td></td><td>Minsk city, Esenina street, 19</td><td>+375(29)643-14-31</td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.894946, 27.555794, 73);" title="Show on map"></div></td><td class="sorting_1">049</td><td></td><td>Minsk city, Kirova street, 8</td><td>+375(29)643-14-31</td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.955007, 27.44644, 45);" title="Show on map"></div></td><td class="sorting_1">05</td><td></td><td>Minsk region, Minsk district, a/g Zhdanovichi, 38</td><td>+375(17)279-88-64</td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.897822, 27.563222, 74);" title="Show on map"></div></td><td class="sorting_1">050</td><td></td><td>Minsk city, Lenina street, 19</td><td>+375(29)643-14-31</td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.863269, 27.454159, 75);" title="Show on map"></div></td><td class="sorting_1">051</td><td></td><td>Minsk city, Lubimova Avenue, 39</td><td>+375(29)643-14-31</td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.894511, 27.476285, 87);" title="Show on map"></div></td><td class="sorting_1">052</td><td></td><td>Minsk city, Ponomarenko street, 10</td><td>+375(29)643-14-31</td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.837423, 27.575557, 76);" title="Show on map"></div></td><td class="sorting_1">053</td><td></td><td>Minsk city, Igumenskiy tract, 45</td><td>+375(29)643-14-31</td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.903204, 27.558784, 77);" title="Show on map"></div></td><td class="sorting_1">054</td><td></td><td>Minsk city, Engelsa street, 10</td><td>+375(29)643-14-31</td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.926886, 27.498105, 78);" title="Show on map"></div></td><td class="sorting_1">055</td><td></td><td>Minsk city, Mihasia Lynkova street, 7</td><td>+375(29)643-14-31</td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.937038, 27.686391, 59);" title="Show on map"></div></td><td class="sorting_1">056</td><td></td><td>Minsk city, Russiyanova street, 13</td><td>+375(29)643-14-31</td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.91548, 27.473284, 79);" title="Show on map"></div></td><td class="sorting_1">057</td><td></td><td>Minsk city, Matusevicha street, 36</td><td>+375(29)643-14-31</td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.914616, 27.493811, 80);" title="Show on map"></div></td><td class="sorting_1">058</td><td></td><td>Minsk city, Pushkina Avenue, 169</td><td>+375(29)643-14-31</td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.909418, 27.485124, 81);" title="Show on map"></div></td><td class="sorting_1">059</td><td></td><td>Minsk city, Pritytskogo street, 10</td><td>+375(29)643-14-31</td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.846685, 27.470298, 36);" title="Show on map"></div></td><td class="sorting_1">06</td><td></td><td>Minsk, Dzerzhinsky Avenue, 126</td><td>+375(17)279-88-64</td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.846476, 27.500728, 60);" title="Show on map"></div></td><td class="sorting_1">060</td><td></td><td>Minsk city, Korzhenevskogo street, 25</td><td>+375(29)643-14-31</td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.864071, 27.487352, 82);" title="Show on map"></div></td><td class="sorting_1">061</td><td></td><td>Minsk city, Dzerzhinskogo Avenue, 73</td><td>+375(29)643-14-31</td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.901292, 27.541376, 89);" title="Show on map"></div></td><td class="sorting_1">062</td><td></td><td>Minsk, Kollektornaia street, 10</td><td>+375(29)643-14-31</td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.903064, 27.503351, 61);" title="Show on map"></div></td><td class="sorting_1">063</td><td></td><td>Minsk city, Boleslava Beruta street, 14</td><td>+375(29)643-14-31</td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.883818, 27.426069, 93);" title="Show on map"></div></td><td class="sorting_1">064</td><td></td><td>Minsk city, Akademika Fedorova street, 5</td><td>+375(29)643-14-31</td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.915491, 27.565972, 92);" title="Show on map"></div></td><td class="sorting_1">065</td><td></td><td>Minsk city, Chicherina street, 21</td><td>+375(29)643-14-31</td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.850571, 27.53878, 83);" title="Show on map"></div></td><td class="sorting_1">066</td><td></td><td>Minsk city, Kizhevatova street, 9</td><td>+375(29)643-14-31</td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.917708, 27.503207, 84);" title="Show on map"></div></td><td class="sorting_1">067</td><td></td><td>Minsk city, Olshevskogo street, 20</td><td>+375(29)643-14-31</td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.898167, 27.52553, 85);" title="Show on map"></div></td><td class="sorting_1">068</td><td></td><td>Minsk city, Voloha street, 1</td><td>+375(29)643-14-31</td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.903514, 27.45618, 62);" title="Show on map"></div></td><td class="sorting_1">069</td><td></td><td>Minsk city, Burdeinogo street, 6</td><td>+375(29)643-14-31</td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.848366, 27.70012, 39);" title="Show on map"></div></td><td class="sorting_1">07</td><td></td><td>Minsk, Partizansky Avenue, 182</td><td>+375(17)279-88-64</td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.909699, 27.578764, 86);" title="Show on map"></div></td><td class="sorting_1">070</td><td></td><td>Minsk city, Kozlova street, 2</td><td>+375(29)643-14-31</td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.955633, 27.620123, 97);" title="Show on map"></div></td><td class="sorting_1">071</td><td></td><td></td><td>+375(29)643-14-31</td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.893959, 27.519934, 102);" title="Show on map"></div></td><td class="sorting_1">072</td><td></td><td>Minsk city, Karla Libknehta street, 79</td><td>+375(29)643-14-31</td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.934748, 27.493586, 103);" title="Show on map"></div></td><td class="sorting_1">073</td><td></td><td>Minsk city, Raduzhnaya street, 27-2</td><td>+375(29)643-14-31</td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.939561, 27.465945, 104);" title="Show on map"></div></td><td class="sorting_1">074</td><td></td><td>Minsk city, Miastrovskaya street, 24</td><td>+375(29)643-14-31</td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.889513, 27.577462, 155);" title="Show on map"></div></td><td class="sorting_1">075</td><td></td><td>Minsk city, Lenina street, 50</td><td>+375(29)643-14-31</td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.83602, 27.604768, 41);" title="Show on map"></div></td><td class="sorting_1">08</td><td></td><td>Minsk, Uborevich street, 176</td><td>+375(17)279-88-64</td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.91179, 27.4136, 40);" title="Show on map"></div></td><td class="sorting_1">09</td><td></td><td>Minsk, Kamennogorskaya street, 3/1</td><td>+375(17)279-88-64</td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(55.152046, 30.127077, 156);" title="Show on map"></div></td><td class="sorting_1">1</td><td></td><td>Vitebskoblnefteproduct, Beshenkovichi highway, 52, 210007, Vitebsk region, Vitebsk</td><td>+375(212)49-35-24</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=6" width="19px" title="Autogas"><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"><img src="/beloil-map/geticon.jpg?t=f&amp;id=168" width="19px" title="BN-95"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=6" width="22px" title="sale of tires"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=13" width="22px" title="parking place"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=90" width="22px" title="AdBlue"><img src="/beloil-map/geticon.jpg?t=s&amp;id=170" width="22px" title="Trailers for rent"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=7" width="22px" title="E100"><img src="/beloil-map/geticon.jpg?t=p&amp;id=12" width="22px" title="DKV"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.939722, 27.571389, 140);" title="Show on map"></div></td><td class="sorting_1">10</td><td></td><td>Minsk city, Nekrasova street, 112</td><td>+375(17)374-98-61</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"><img src="/beloil-map/geticon.jpg?t=f&amp;id=128" width="19px" title="RON-98"><img src="/beloil-map/geticon.jpg?t=f&amp;id=188" width="19px" title="AI-95 Flagman"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=6" width="22px" title="sale of tires"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=13" width="22px" title="parking place"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.723726, 27.504024, 118);" title="Show on map"></div></td><td class="sorting_1">14</td><td></td><td>motorway Brest - Border of the Russian Federation, 346 km (left)</td><td>+375(17)511-42-56</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=6" width="19px" title="Autogas"><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"><img src="/beloil-map/geticon.jpg?t=f&amp;id=188" width="19px" title="AI-95 Flagman"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=6" width="22px" title="sale of tires"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=13" width="22px" title="parking place"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=90" width="22px" title="AdBlue"><img src="/beloil-map/geticon.jpg?t=s&amp;id=150" width="22px" title="toll road services"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=350" width="22px" title="Baby changing table"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=7" width="22px" title="E100"><img src="/beloil-map/geticon.jpg?t=p&amp;id=12" width="22px" title="DKV"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=169" width="22px" title="CRT "><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(54.234534, 25.357986, 99);" title="Show on map"></div></td><td class="sorting_1">17</td><td></td><td>Lidanefteproduct,4-th km(1) M-11 231381, Voronovo district, Grodno region, Belarus</td><td>+375(1594)4-09-81</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=1" width="19px" title="RON-95"><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=13" width="22px" title="parking place"><img src="/beloil-map/geticon.jpg?t=s&amp;id=17" width="22px" title="insurance services"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=90" width="22px" title="AdBlue"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=230" width="22px" title="Caravan parking"><img src="/beloil-map/geticon.jpg?t=s&amp;id=350" width="22px" title="Baby changing table"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=7" width="22px" title="E100"><img src="/beloil-map/geticon.jpg?t=p&amp;id=8" width="22px" title="Petrol Plus Region"><img src="/beloil-map/geticon.jpg?t=p&amp;id=9" width="22px" title="Autoil"><img src="/beloil-map/geticon.jpg?t=p&amp;id=12" width="22px" title="DKV"><img src="/beloil-map/geticon.jpg?t=p&amp;id=28" width="22px" title="Aris"><img src="/beloil-map/geticon.jpg?t=p&amp;id=69" width="22px" title="Tank You Baltics"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=169" width="22px" title="CRT "><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(54.91141, 26.700411, 167);" title="Show on map"></div></td><td class="sorting_1">17</td><td></td><td>Myadel district, Naroch, st. Oktyabrskaya, 35</td><td>+375(1797)2-90-20</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=1" width="19px" title="RON-95"><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=13" width="22px" title="parking place"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=330" width="22px" title="Bicycles for rent"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(52.468056, 30.974444, 112);" title="Show on map"></div></td><td class="sorting_1">18</td><td></td><td>Gomel region, Gomel district, Gomel city, Geroyev Podpol'shchikov str.20</td><td>+375(232)56-73-35</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"><img src="/beloil-map/geticon.jpg?t=f&amp;id=128" width="19px" title="RON-98"><img src="/beloil-map/geticon.jpg?t=f&amp;id=188" width="19px" title="AI-95 Flagman"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=13" width="22px" title="parking place"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(52.5544, 23.7951, 129);" title="Show on map"></div></td><td class="sorting_1">18</td><td></td><td>Brestoblnefteproduct; Street Pushyanskaya 2A; 225063, Kamenec district, Brest region, Belarus, Kamenyuki agro-town</td><td>+375(1631)5-62-52</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=1" width="19px" title="RON-95"><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=230" width="22px" title="Caravan parking"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.9341, 30.33423, 162);" title="Show on map"></div></td><td class="sorting_1">2</td><td></td><td>Mogilev region, Mogilev district, Mogilev city, Pervomaiskay street,146</td><td>+375(222)63-02-72</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=1" width="19px" title="RON-95"><img src="/beloil-map/geticon.jpg?t=f&amp;id=6" width="19px" title="Autogas"><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=6" width="22px" title="sale of tires"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=26" width="22px" title="Refilling the washer"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(52.443638, 31.023259, 125);" title="Show on map"></div></td><td class="sorting_1">20</td><td></td><td>Gomel region, Gomel district, Gomel city,  Khataevicha str. 40</td><td>+375(232)51-35-56</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"><img src="/beloil-map/geticon.jpg?t=f&amp;id=188" width="19px" title="AI-95 Flagman"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=13" width="22px" title="parking place"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=90" width="22px" title="AdBlue"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.844658, 25.183266, 159);" title="Show on map"></div></td><td class="sorting_1">21</td><td></td><td>Lidanefteprodukt, 172-th km,M-6,231308, Lida district, Grodno region,Belarus.</td><td>+375(154)58-03-25</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=1" width="19px" title="RON-95"><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=6" width="22px" title="sale of tires"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=13" width="22px" title="parking place"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=90" width="22px" title="AdBlue"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=350" width="22px" title="Baby changing table"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=7" width="22px" title="E100"><img src="/beloil-map/geticon.jpg?t=p&amp;id=8" width="22px" title="Petrol Plus Region"><img src="/beloil-map/geticon.jpg?t=p&amp;id=12" width="22px" title="DKV"><img src="/beloil-map/geticon.jpg?t=p&amp;id=28" width="22px" title="Aris"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=169" width="22px" title="CRT "><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.884392, 27.562536, 119);" title="Show on map"></div></td><td class="sorting_1">22</td><td></td><td>Minsk city, Volodko street, 3</td><td>+375(17)216-24-40</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"><img src="/beloil-map/geticon.jpg?t=f&amp;id=128" width="19px" title="RON-98"><img src="/beloil-map/geticon.jpg?t=f&amp;id=188" width="19px" title="AI-95 Flagman"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=13" width="22px" title="parking place"><img src="/beloil-map/geticon.jpg?t=s&amp;id=17" width="22px" title="insurance services"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=330" width="22px" title="Bicycles for rent"><img src="/beloil-map/geticon.jpg?t=s&amp;id=350" width="22px" title="Baby changing table"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.949722, 27.447222, 120);" title="Show on map"></div></td><td class="sorting_1">23</td><td></td><td>Minsk city, Pobediteley Avenue, 143</td><td>+375(17)310-53-04</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"><img src="/beloil-map/geticon.jpg?t=f&amp;id=128" width="19px" title="RON-98"><img src="/beloil-map/geticon.jpg?t=f&amp;id=188" width="19px" title="AI-95 Flagman"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=330" width="22px" title="Bicycles for rent"><img src="/beloil-map/geticon.jpg?t=s&amp;id=350" width="22px" title="Baby changing table"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.533611, 26.764689, 165);" title="Show on map"></div></td><td class="sorting_1">25</td><td></td><td>Minsk region, Stolbtsovskiy  district, motorway M-1, 288 km, 1</td><td>+375(1717)5-31-84</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"><img src="/beloil-map/geticon.jpg?t=f&amp;id=188" width="19px" title="AI-95 Flagman"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=13" width="22px" title="parking place"><img src="/beloil-map/geticon.jpg?t=s&amp;id=90" width="22px" title="AdBlue"><img src="/beloil-map/geticon.jpg?t=s&amp;id=150" width="22px" title="toll road services"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=350" width="22px" title="Baby changing table"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=7" width="22px" title="E100"><img src="/beloil-map/geticon.jpg?t=p&amp;id=12" width="22px" title="DKV"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=169" width="22px" title="CRT "><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.843055, 27.559166, 150);" title="Show on map"></div></td><td class="sorting_1">27</td><td></td><td>Minsk city, V.Syrokomli street, 9</td><td>+375(17)249-74-56</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=6" width="19px" title="Autogas"><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"><img src="/beloil-map/geticon.jpg?t=f&amp;id=188" width="19px" title="AI-95 Flagman"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=6" width="22px" title="sale of tires"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=13" width="22px" title="parking place"><img src="/beloil-map/geticon.jpg?t=s&amp;id=90" width="22px" title="AdBlue"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=330" width="22px" title="Bicycles for rent"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.723056, 27.505278, 113);" title="Show on map"></div></td><td class="sorting_1">28</td><td></td><td>motorway Brest - border of the Russian Federation M-1 346 km (right), the village area of ​​Ugly</td><td>+375(17)506-67-59</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"><img src="/beloil-map/geticon.jpg?t=f&amp;id=188" width="19px" title="AI-95 Flagman"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=13" width="22px" title="parking place"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=90" width="22px" title="AdBlue"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=350" width="22px" title="Baby changing table"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=7" width="22px" title="E100"><img src="/beloil-map/geticon.jpg?t=p&amp;id=12" width="22px" title="DKV"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=169" width="22px" title="CRT "><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(52.101615, 30.971571, 141);" title="Show on map"></div></td><td class="sorting_1">28</td><td></td><td>Gomel region, Gomel district, Teryukhskiy Sel'skiy Sovet, 18, near agrotown Novaya Guta</td><td>+375(232)92-98-73</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=6" width="19px" title="Autogas"><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"><img src="/beloil-map/geticon.jpg?t=f&amp;id=128" width="19px" title="RON-98"><img src="/beloil-map/geticon.jpg?t=f&amp;id=188" width="19px" title="AI-95 Flagman"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=6" width="22px" title="sale of tires"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=17" width="22px" title="insurance services"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=90" width="22px" title="AdBlue"><img src="/beloil-map/geticon.jpg?t=s&amp;id=150" width="22px" title="toll road services"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=230" width="22px" title="Caravan parking"><img src="/beloil-map/geticon.jpg?t=s&amp;id=270" width="22px" title="Adblue in bulk"><img src="/beloil-map/geticon.jpg?t=s&amp;id=350" width="22px" title="Baby changing table"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=7" width="22px" title="E100"><img src="/beloil-map/geticon.jpg?t=p&amp;id=12" width="22px" title="DKV"><img src="/beloil-map/geticon.jpg?t=p&amp;id=28" width="22px" title="Aris"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=169" width="22px" title="CRT "><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.191667, 26.075278, 114);" title="Show on map"></div></td><td class="sorting_1">29</td><td></td><td>Brest region, Baranovichi district, 226-th km of Brest highway, 3</td><td>+375(163)65-76-24</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=1" width="19px" title="RON-95"><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=90" width="22px" title="AdBlue"><img src="/beloil-map/geticon.jpg?t=s&amp;id=150" width="22px" title="toll road services"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=350" width="22px" title="Baby changing table"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=7" width="22px" title="E100"><img src="/beloil-map/geticon.jpg?t=p&amp;id=12" width="22px" title="DKV"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=169" width="22px" title="CRT "><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.215192, 29.083108, 158);" title="Show on map"></div></td><td class="sorting_1">29</td><td></td><td>Mogilev region, Bobruisk district, track M-5, 131 km, 2</td><td>+375(225)75-90-07</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=1" width="19px" title="RON-95"><img src="/beloil-map/geticon.jpg?t=f&amp;id=6" width="19px" title="Autogas"><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=13" width="22px" title="parking place"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=26" width="22px" title="Refilling the washer"><img src="/beloil-map/geticon.jpg?t=s&amp;id=90" width="22px" title="AdBlue"><img src="/beloil-map/geticon.jpg?t=s&amp;id=150" width="22px" title="toll road services"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=350" width="22px" title="Baby changing table"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=7" width="22px" title="E100"><img src="/beloil-map/geticon.jpg?t=p&amp;id=12" width="22px" title="DKV"><img src="/beloil-map/geticon.jpg?t=p&amp;id=69" width="22px" title="Tank You Baltics"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=169" width="22px" title="CRT "><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(55.225734, 30.115542, 157);" title="Show on map"></div></td><td class="sorting_1">3</td><td></td><td>Vitebskoblnefteproduct, Leningradskaja street, 178a, 210015, Vitebsk region, Vitebsk, Belarus</td><td>+375(212)65-90-40</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=6" width="19px" title="Autogas"><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"><img src="/beloil-map/geticon.jpg?t=f&amp;id=168" width="19px" title="BN-95"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=6" width="22px" title="sale of tires"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=90" width="22px" title="AdBlue"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=12" width="22px" title="DKV"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.838286, 25.344623, 161);" title="Show on map"></div></td><td class="sorting_1">3</td><td></td><td>Agro-town Jodki, 124, 231310, Lida district, Grodno region, Tret'jakovskij village council, Belarus</td><td>+375(154)62-60-34</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=1" width="19px" title="RON-95"><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=13" width="22px" title="parking place"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=90" width="22px" title="AdBlue"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=350" width="22px" title="Baby changing table"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=7" width="22px" title="E100"><img src="/beloil-map/geticon.jpg?t=p&amp;id=12" width="22px" title="DKV"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=169" width="22px" title="CRT "><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.817929, 27.428333, 149);" title="Show on map"></div></td><td class="sorting_1">30</td><td></td><td>Minsk region, motorway Minsk-Brest R-1, 12th km, 2</td><td>+375(17)542-09-12</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=6" width="19px" title="Autogas"><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"><img src="/beloil-map/geticon.jpg?t=f&amp;id=188" width="19px" title="AI-95 Flagman"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=6" width="22px" title="sale of tires"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=13" width="22px" title="parking place"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=90" width="22px" title="AdBlue"><img src="/beloil-map/geticon.jpg?t=s&amp;id=170" width="22px" title="Trailers for rent"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=330" width="22px" title="Bicycles for rent"><img src="/beloil-map/geticon.jpg?t=s&amp;id=350" width="22px" title="Baby changing table"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=12" width="22px" title="DKV"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=169" width="22px" title="CRT "><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.141944, 24.803056, 153);" title="Show on map"></div></td><td class="sorting_1">31</td><td></td><td>Grodno region, town Zelva, str. Shossejnaja, 1</td><td>+375(1564)7-46-13</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=1" width="19px" title="RON-95"><img src="/beloil-map/geticon.jpg?t=f&amp;id=6" width="19px" title="Autogas"><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=7" width="22px" title="E100"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(54.884338, 28.728319, 105);" title="Show on map"></div></td><td class="sorting_1">33</td><td></td><td>Vitebskoblnefteprodykt,Respublika Belarus,Vitebski region,211180 Lepel sity, Cyikova street ,199</td><td>+375(2132)3-36-53</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=6" width="19px" title="Autogas"><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"><img src="/beloil-map/geticon.jpg?t=f&amp;id=168" width="19px" title="BN-95"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=6" width="22px" title="sale of tires"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=13" width="22px" title="parking place"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=90" width="22px" title="AdBlue"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=350" width="22px" title="Baby changing table"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=7" width="22px" title="E100"><img src="/beloil-map/geticon.jpg?t=p&amp;id=12" width="22px" title="DKV"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=169" width="22px" title="CRT "><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.893278, 30.325111, 100);" title="Show on map"></div></td><td class="sorting_1">37</td><td></td><td>Mogilev region, Mogilev district, Mogilev city, Chelyuskintsev Street, 8</td><td>+375(222)78-33-49</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=1" width="19px" title="RON-95"><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=13" width="22px" title="parking place"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(54.051493, 26.770348, 168);" title="Show on map"></div></td><td class="sorting_1">38</td><td></td><td>57 km road Minsk-Grodno</td><td>+375(1772)6-37-37</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=6" width="19px" title="Autogas"><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"><img src="/beloil-map/geticon.jpg?t=f&amp;id=188" width="19px" title="AI-95 Flagman"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=17" width="22px" title="insurance services"><img src="/beloil-map/geticon.jpg?t=s&amp;id=150" width="22px" title="toll road services"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=270" width="22px" title="Adblue in bulk"><img src="/beloil-map/geticon.jpg?t=s&amp;id=350" width="22px" title="Baby changing table"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=7" width="22px" title="E100"><img src="/beloil-map/geticon.jpg?t=p&amp;id=12" width="22px" title="DKV"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=169" width="22px" title="CRT "><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(52.406157, 30.917856, 123);" title="Show on map"></div></td><td class="sorting_1">39</td><td></td><td>Gomel city, Kosareva str. 20</td><td>+375(232)50-26-73</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=6" width="19px" title="Autogas"><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"><img src="/beloil-map/geticon.jpg?t=f&amp;id=188" width="19px" title="AI-95 Flagman"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=10" width="22px" title="Car wash"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=13" width="22px" title="parking place"><img src="/beloil-map/geticon.jpg?t=s&amp;id=170" width="22px" title="Trailers for rent"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.858055, 27.678333, 117);" title="Show on map"></div></td><td class="sorting_1">4</td><td></td><td>Minsk city, Partizansky Avenue, 156a</td><td>+375(17)373-20-74</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"><img src="/beloil-map/geticon.jpg?t=f&amp;id=188" width="19px" title="AI-95 Flagman"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=6" width="22px" title="sale of tires"><img src="/beloil-map/geticon.jpg?t=s&amp;id=10" width="22px" title="Car wash"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=170" width="22px" title="Trailers for rent"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=350" width="22px" title="Baby changing table"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=12" width="22px" title="DKV"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=169" width="22px" title="CRT "><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(52.0536, 23.7234, 127);" title="Show on map"></div></td><td class="sorting_1">4</td><td></td><td>Brestoblnefteprodukt; Kovel highway 105; 224002, Brest region, Brest, Belarus</td><td>+375(162)52-67-95</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=1" width="19px" title="RON-95"><img src="/beloil-map/geticon.jpg?t=f&amp;id=6" width="19px" title="Autogas"><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=6" width="22px" title="sale of tires"><img src="/beloil-map/geticon.jpg?t=s&amp;id=7" width="22px" title="Service station"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=16" width="22px" title="shower for customers"><img src="/beloil-map/geticon.jpg?t=s&amp;id=90" width="22px" title="AdBlue"><img src="/beloil-map/geticon.jpg?t=s&amp;id=170" width="22px" title="Trailers for rent"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=7" width="22px" title="E100"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.871083, 30.355694, 154);" title="Show on map"></div></td><td class="sorting_1">40</td><td></td><td>Mogilev region, Mogilev district, Mogilev city, Gabrovskay Street, 39</td><td>+375(222)74-22-92</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=1" width="19px" title="RON-95"><img src="/beloil-map/geticon.jpg?t=f&amp;id=6" width="19px" title="Autogas"><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=6" width="22px" title="sale of tires"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(52.124444, 29.304167, 115);" title="Show on map"></div></td><td class="sorting_1">43</td><td></td><td>Kalinkovichi city, 50 let Oktyabrya str., 54</td><td>+375(2345)2-37-69</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"><img src="/beloil-map/geticon.jpg?t=f&amp;id=188" width="19px" title="AI-95 Flagman"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=6" width="22px" title="sale of tires"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=170" width="22px" title="Trailers for rent"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(52.550278, 25.063611, 163);" title="Show on map"></div></td><td class="sorting_1">44</td><td></td><td>Brest region, Beryoza district, 123-th km of Brest highway, 2</td><td>+375(1643)5-52-81</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=1" width="19px" title="RON-95"><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=6" width="22px" title="sale of tires"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=13" width="22px" title="parking place"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=90" width="22px" title="AdBlue"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=230" width="22px" title="Caravan parking"><img src="/beloil-map/geticon.jpg?t=s&amp;id=350" width="22px" title="Baby changing table"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=7" width="22px" title="E100"><img src="/beloil-map/geticon.jpg?t=p&amp;id=12" width="22px" title="DKV"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=169" width="22px" title="CRT "><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.952222, 27.713055, 121);" title="Show on map"></div></td><td class="sorting_1">45</td><td></td><td>Minsk city, Nezavisimosti  Avenue, 200</td><td>+375(17)363-87-78</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"><img src="/beloil-map/geticon.jpg?t=f&amp;id=128" width="19px" title="RON-98"><img src="/beloil-map/geticon.jpg?t=f&amp;id=188" width="19px" title="AI-95 Flagman"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=6" width="22px" title="sale of tires"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=13" width="22px" title="parking place"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=90" width="22px" title="AdBlue"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=330" width="22px" title="Bicycles for rent"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(52.984169, 27.503611, 166);" title="Show on map"></div></td><td class="sorting_1">49</td><td></td><td>Minsk region, Slutskiy district, Kirovskiy village council, motorway R-23, 109 km, 1</td><td>+375(1795)3-19-21</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=6" width="19px" title="Autogas"><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"><img src="/beloil-map/geticon.jpg?t=f&amp;id=188" width="19px" title="AI-95 Flagman"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=90" width="22px" title="AdBlue"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.829326, 28.984122, 138);" title="Show on map"></div></td><td class="sorting_1">5</td><td></td><td>Minsk region, Berezinsky borough, city of Berezino, Maksim Gorkogo st., 50</td><td>+375(1715)5-45-12</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"><img src="/beloil-map/geticon.jpg?t=f&amp;id=188" width="19px" title="AI-95 Flagman"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=6" width="22px" title="sale of tires"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=13" width="22px" title="parking place"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=150" width="22px" title="toll road services"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=270" width="22px" title="Adblue in bulk"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=7" width="22px" title="E100"><img src="/beloil-map/geticon.jpg?t=p&amp;id=12" width="22px" title="DKV"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(54.62716, 30.43605, 106);" title="Show on map"></div></td><td class="sorting_1">52</td><td></td><td>Vitebskoblnefteproduct, 211024, Orshansky district, Vitebsk region, Vysokoe village, Belarus</td><td>+375(21)627-95-08</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=6" width="19px" title="Autogas"><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"><img src="/beloil-map/geticon.jpg?t=f&amp;id=128" width="19px" title="RON-98"><img src="/beloil-map/geticon.jpg?t=f&amp;id=168" width="19px" title="BN-95"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=6" width="22px" title="sale of tires"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=13" width="22px" title="parking place"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=150" width="22px" title="toll road services"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=230" width="22px" title="Caravan parking"><img src="/beloil-map/geticon.jpg?t=s&amp;id=270" width="22px" title="Adblue in bulk"><img src="/beloil-map/geticon.jpg?t=s&amp;id=350" width="22px" title="Baby changing table"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=7" width="22px" title="E100"><img src="/beloil-map/geticon.jpg?t=p&amp;id=12" width="22px" title="DKV"><img src="/beloil-map/geticon.jpg?t=p&amp;id=169" width="22px" title="CRT "><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(54.624752, 30.453122, 107);" title="Show on map"></div></td><td class="sorting_1">53</td><td></td><td>Vitebskoblnefteproduct, 575-th km of Brest-Minsk-Russian border road, 211024, Orshansky district, Vitebsk region, Belarus</td><td>+375(216)27-95-09</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"><img src="/beloil-map/geticon.jpg?t=f&amp;id=128" width="19px" title="RON-98"><img src="/beloil-map/geticon.jpg?t=f&amp;id=168" width="19px" title="BN-95"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=13" width="22px" title="parking place"><img src="/beloil-map/geticon.jpg?t=s&amp;id=17" width="22px" title="insurance services"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=90" width="22px" title="AdBlue"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=270" width="22px" title="Adblue in bulk"><img src="/beloil-map/geticon.jpg?t=s&amp;id=350" width="22px" title="Baby changing table"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=7" width="22px" title="E100"><img src="/beloil-map/geticon.jpg?t=p&amp;id=12" width="22px" title="DKV"><img src="/beloil-map/geticon.jpg?t=p&amp;id=169" width="22px" title="CRT "><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.968429, 27.526385, 139);" title="Show on map"></div></td><td class="sorting_1">57</td><td></td><td>Minsk city, Dolginovskiy Trakt, 189</td><td>+375(17)335-34-22</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"><img src="/beloil-map/geticon.jpg?t=f&amp;id=128" width="19px" title="RON-98"><img src="/beloil-map/geticon.jpg?t=f&amp;id=188" width="19px" title="AI-95 Flagman"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=6" width="22px" title="sale of tires"><img src="/beloil-map/geticon.jpg?t=s&amp;id=10" width="22px" title="Car wash"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=90" width="22px" title="AdBlue"><img src="/beloil-map/geticon.jpg?t=s&amp;id=170" width="22px" title="Trailers for rent"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=350" width="22px" title="Baby changing table"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(54.43475, 29.6044, 108);" title="Show on map"></div></td><td class="sorting_1">59</td><td></td><td>Vitebskoblnefteproduct, 211070, Tolochinsky district, Vitebsk region, Myravnichi village, Belarus</td><td>+375(2136)2-67-59</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"><img src="/beloil-map/geticon.jpg?t=f&amp;id=168" width="19px" title="BN-95"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=17" width="22px" title="insurance services"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=270" width="22px" title="Adblue in bulk"><img src="/beloil-map/geticon.jpg?t=s&amp;id=350" width="22px" title="Baby changing table"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=7" width="22px" title="E100"><img src="/beloil-map/geticon.jpg?t=p&amp;id=12" width="22px" title="DKV"><img src="/beloil-map/geticon.jpg?t=p&amp;id=169" width="22px" title="CRT "><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(52.492828, 30.687603, 124);" title="Show on map"></div></td><td class="sorting_1">59</td><td></td><td>Gomel region, Gomel district, track M-5, 285-th km, 2</td><td>+375(44)599-28-30</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"><img src="/beloil-map/geticon.jpg?t=f&amp;id=188" width="19px" title="AI-95 Flagman"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=13" width="22px" title="parking place"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=90" width="22px" title="AdBlue"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=350" width="22px" title="Baby changing table"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=7" width="22px" title="E100"><img src="/beloil-map/geticon.jpg?t=p&amp;id=12" width="22px" title="DKV"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=169" width="22px" title="CRT "><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(52.0771, 23.7235, 128);" title="Show on map"></div></td><td class="sorting_1">6</td><td></td><td>Brestoblnefteprodukt; Warsaw highway 3; 224003, Brest region, Brest, Belarus</td><td>+375(162)59-12-94</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=1" width="19px" title="RON-95"><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=6" width="22px" title="sale of tires"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=13" width="22px" title="parking place"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=150" width="22px" title="toll road services"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=350" width="22px" title="Baby changing table"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=12" width="22px" title="DKV"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=169" width="22px" title="CRT "><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.964446, 28.025854, 148);" title="Show on map"></div></td><td class="sorting_1">6</td><td></td><td>Minsk region, Smolevichsky district, M-1, 391 km, 2</td><td>+375(1776)2-15-79</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"><img src="/beloil-map/geticon.jpg?t=f&amp;id=188" width="19px" title="AI-95 Flagman"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=13" width="22px" title="parking place"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=90" width="22px" title="AdBlue"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=350" width="22px" title="Baby changing table"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=7" width="22px" title="E100"><img src="/beloil-map/geticon.jpg?t=p&amp;id=12" width="22px" title="DKV"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=169" width="22px" title="CRT "><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.961111, 27.625315, 160);" title="Show on map"></div></td><td class="sorting_1">6</td><td></td><td>​​Minsk city, Logoyskiy trakt, 49</td><td>+375(17)259-62-64</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=6" width="19px" title="Autogas"><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"><img src="/beloil-map/geticon.jpg?t=f&amp;id=128" width="19px" title="RON-98"><img src="/beloil-map/geticon.jpg?t=f&amp;id=188" width="19px" title="AI-95 Flagman"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=350" width="22px" title="Baby changing table"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=69" width="22px" title="Tank You Baltics"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(54.4367, 29.60463, 109);" title="Show on map"></div></td><td class="sorting_1">60</td><td></td><td>Vitebskoblnefteproduct, 211070, Tolochinsky district, Vitebsk region, Myravnichi village, Belarus</td><td>+375(2136)2-67-60</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"><img src="/beloil-map/geticon.jpg?t=f&amp;id=128" width="19px" title="RON-98"><img src="/beloil-map/geticon.jpg?t=f&amp;id=168" width="19px" title="BN-95"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=90" width="22px" title="AdBlue"><img src="/beloil-map/geticon.jpg?t=s&amp;id=150" width="22px" title="toll road services"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=350" width="22px" title="Baby changing table"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=7" width="22px" title="E100"><img src="/beloil-map/geticon.jpg?t=p&amp;id=12" width="22px" title="DKV"><img src="/beloil-map/geticon.jpg?t=p&amp;id=169" width="22px" title="CRT "><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.889382, 27.692291, 122);" title="Show on map"></div></td><td class="sorting_1">62</td><td></td><td>Minsk city, Ilimskaya street, 6G</td><td>+375(17)361-41-24</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"><img src="/beloil-map/geticon.jpg?t=f&amp;id=188" width="19px" title="AI-95 Flagman"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=13" width="22px" title="parking place"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=90" width="22px" title="AdBlue"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=330" width="22px" title="Bicycles for rent"><img src="/beloil-map/geticon.jpg?t=s&amp;id=350" width="22px" title="Baby changing table"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.906433, 29.399433, 146);" title="Show on map"></div></td><td class="sorting_1">67</td><td></td><td>Mogilev region,  Belynichi district, track M-4, 132 km,1</td><td>+375(2232)5-00-32</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=1" width="19px" title="RON-95"><img src="/beloil-map/geticon.jpg?t=f&amp;id=6" width="19px" title="Autogas"><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=13" width="22px" title="parking place"><img src="/beloil-map/geticon.jpg?t=s&amp;id=16" width="22px" title="shower for customers"><img src="/beloil-map/geticon.jpg?t=s&amp;id=18" width="22px" title="hotel"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=26" width="22px" title="Refilling the washer"><img src="/beloil-map/geticon.jpg?t=s&amp;id=90" width="22px" title="AdBlue"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=230" width="22px" title="Caravan parking"><img src="/beloil-map/geticon.jpg?t=s&amp;id=350" width="22px" title="Baby changing table"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=7" width="22px" title="E100"><img src="/beloil-map/geticon.jpg?t=p&amp;id=12" width="22px" title="DKV"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=169" width="22px" title="CRT "><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.91487, 30.4712, 57);" title="Show on map"></div></td><td class="sorting_1">68</td><td></td><td>Mogilev region,  Mogilev district, track M-8, 232 km,1</td><td>+375(222)66-91-26</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=1" width="19px" title="RON-95"><img src="/beloil-map/geticon.jpg?t=f&amp;id=6" width="19px" title="Autogas"><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=26" width="22px" title="Refilling the washer"><img src="/beloil-map/geticon.jpg?t=s&amp;id=90" width="22px" title="AdBlue"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=350" width="22px" title="Baby changing table"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=7" width="22px" title="E100"><img src="/beloil-map/geticon.jpg?t=p&amp;id=12" width="22px" title="DKV"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=169" width="22px" title="CRT "><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.9094, 29.4112, 111);" title="Show on map"></div></td><td class="sorting_1">69</td><td></td><td>Mogilev region,  Belynichi district, track M-4, 132 km,3</td><td>+375(2232)5-00-41</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=1" width="19px" title="RON-95"><img src="/beloil-map/geticon.jpg?t=f&amp;id=6" width="19px" title="Autogas"><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=13" width="22px" title="parking place"><img src="/beloil-map/geticon.jpg?t=s&amp;id=16" width="22px" title="shower for customers"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=26" width="22px" title="Refilling the washer"><img src="/beloil-map/geticon.jpg?t=s&amp;id=90" width="22px" title="AdBlue"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=350" width="22px" title="Baby changing table"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=12" width="22px" title="DKV"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=169" width="22px" title="CRT "><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.620225, 28.05086, 144);" title="Show on map"></div></td><td class="sorting_1">7</td><td></td><td>road Minsk-Gomel, 47-km (left)</td><td>+375(1713)6-65-65</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"><img src="/beloil-map/geticon.jpg?t=f&amp;id=188" width="19px" title="AI-95 Flagman"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=13" width="22px" title="parking place"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=90" width="22px" title="AdBlue"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=350" width="22px" title="Baby changing table"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=7" width="22px" title="E100"><img src="/beloil-map/geticon.jpg?t=p&amp;id=12" width="22px" title="DKV"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=169" width="22px" title="CRT "><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(55.474099, 28.745358, 98);" title="Show on map"></div></td><td class="sorting_1">71</td><td></td><td>Vitebskoblnefteprodykt,Respublika Belarus,Vitebski region,211440 Polatski district,Polotsk town, Vilniskae highway.2</td><td>+375(214)46-88-14</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"><img src="/beloil-map/geticon.jpg?t=f&amp;id=168" width="19px" title="BN-95"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=10" width="22px" title="Car wash"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=170" width="22px" title="Trailers for rent"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=7" width="22px" title="E100"><img src="/beloil-map/geticon.jpg?t=p&amp;id=69" width="22px" title="Tank You Baltics"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(52.2426, 27.4523, 116);" title="Show on map"></div></td><td class="sorting_1">74</td><td></td><td>Brestoblnefteprodukt, Brest region, Luninetsky district, 0.9 km to the north-west from the city of Mikashevichi</td><td>+375(164)75-53-24</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=1" width="19px" title="RON-95"><img src="/beloil-map/geticon.jpg?t=f&amp;id=6" width="19px" title="Autogas"><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"><img src="/beloil-map/geticon.jpg?t=f&amp;id=188" width="19px" title="AI-95 Flagman"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=6" width="22px" title="sale of tires"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=13" width="22px" title="parking place"><img src="/beloil-map/geticon.jpg?t=s&amp;id=16" width="22px" title="shower for customers"><img src="/beloil-map/geticon.jpg?t=s&amp;id=17" width="22px" title="insurance services"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=90" width="22px" title="AdBlue"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=7" width="22px" title="E100"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.923028, 27.742625, 169);" title="Show on map"></div></td><td class="sorting_1">74</td><td></td><td>Minsk region Minsk Street. Tankovaya,8</td><td>+375(17)514-61-04</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"><img src="/beloil-map/geticon.jpg?t=f&amp;id=188" width="19px" title="AI-95 Flagman"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=13" width="22px" title="parking place"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=90" width="22px" title="AdBlue"><img src="/beloil-map/geticon.jpg?t=s&amp;id=170" width="22px" title="Trailers for rent"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=350" width="22px" title="Baby changing table"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(54.375383, 29.3149, 143);" title="Show on map"></div></td><td class="sorting_1">76</td><td></td><td>M-1,494-th km, 76</td><td>+375(1796)4-62-95</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=1" width="19px" title="RON-95"><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=90" width="22px" title="AdBlue"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=350" width="22px" title="Baby changing table"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=7" width="22px" title="E100"><img src="/beloil-map/geticon.jpg?t=p&amp;id=12" width="22px" title="DKV"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=169" width="22px" title="CRT "><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(52.1568, 25.5597, 164);" title="Show on map"></div></td><td class="sorting_1">76</td><td></td><td>Brest region, Ivanovo R-n, Lastovetsky s/s, M-10, 438 km, 10</td><td>+375(1652)2-69-09</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=1" width="19px" title="RON-95"><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=6" width="22px" title="sale of tires"><img src="/beloil-map/geticon.jpg?t=s&amp;id=8" width="22px" title="tire fitting shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=13" width="22px" title="parking place"><img src="/beloil-map/geticon.jpg?t=s&amp;id=16" width="22px" title="shower for customers"><img src="/beloil-map/geticon.jpg?t=s&amp;id=17" width="22px" title="insurance services"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=90" width="22px" title="AdBlue"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=7" width="22px" title="E100"><img src="/beloil-map/geticon.jpg?t=p&amp;id=12" width="22px" title="DKV"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=169" width="22px" title="CRT "><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(55.16386, 30.284694, 101);" title="Show on map"></div></td><td class="sorting_1">78</td><td></td><td>Vitebskoblnefteproduct, 33 Armii strit, 24, 210038, Vitebsk region, Vitebsk</td><td>+375(212)37-77-78</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"><img src="/beloil-map/geticon.jpg?t=f&amp;id=128" width="19px" title="RON-98"><img src="/beloil-map/geticon.jpg?t=f&amp;id=168" width="19px" title="BN-95"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.155437, 30.460582, 142);" title="Show on map"></div></td><td class="sorting_1">78</td><td></td><td>Gomel region, Rogachev district, Dovskiy Sel'skiy Sovet, Dovsk agrotown, Gomel'skaya str. 2B</td><td>+375(2339)7-73-07</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=6" width="19px" title="Autogas"><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"><img src="/beloil-map/geticon.jpg?t=f&amp;id=188" width="19px" title="AI-95 Flagman"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=6" width="22px" title="sale of tires"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=90" width="22px" title="AdBlue"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=7" width="22px" title="E100"><img src="/beloil-map/geticon.jpg?t=p&amp;id=12" width="22px" title="DKV"><img src="/beloil-map/geticon.jpg?t=p&amp;id=28" width="22px" title="Aris"><img src="/beloil-map/geticon.jpg?t=p&amp;id=69" width="22px" title="Tank You Baltics"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=169" width="22px" title="CRT "><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.80534, 27.798022, 152);" title="Show on map"></div></td><td class="sorting_1">78</td><td></td><td>Minsk region, Minsk district, M4 19 km, 3</td><td>+375(17)515-40-76</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"><img src="/beloil-map/geticon.jpg?t=f&amp;id=128" width="19px" title="RON-98"><img src="/beloil-map/geticon.jpg?t=f&amp;id=188" width="19px" title="AI-95 Flagman"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=13" width="22px" title="parking place"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=90" width="22px" title="AdBlue"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=270" width="22px" title="Adblue in bulk"><img src="/beloil-map/geticon.jpg?t=s&amp;id=350" width="22px" title="Baby changing table"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(54.44, 25.965, 145);" title="Show on map"></div></td><td class="sorting_1">81</td><td></td><td>127-th km of the М-7 highway, 4, 231103, Oshmyany district, Grodno region, Belarus</td><td>+375(1593)7-31-28</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"><img src="/beloil-map/geticon.jpg?t=f&amp;id=168" width="19px" title="BN-95"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=6" width="22px" title="sale of tires"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=13" width="22px" title="parking place"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=90" width="22px" title="AdBlue"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=350" width="22px" title="Baby changing table"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=7" width="22px" title="E100"><img src="/beloil-map/geticon.jpg?t=p&amp;id=8" width="22px" title="Petrol Plus Region"><img src="/beloil-map/geticon.jpg?t=p&amp;id=12" width="22px" title="DKV"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=169" width="22px" title="CRT "><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(52.3676, 31.003, 110);" title="Show on map"></div></td><td class="sorting_1">83</td><td></td><td>Gomel region, Gomel district, track M-150, 4-th km, 1</td><td>+375(232)25-06-94</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"><img src="/beloil-map/geticon.jpg?t=f&amp;id=128" width="19px" title="RON-98"><img src="/beloil-map/geticon.jpg?t=f&amp;id=188" width="19px" title="AI-95 Flagman"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=13" width="22px" title="parking place"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=170" width="22px" title="Trailers for rent"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.531543, 23.683868, 151);" title="Show on map"></div></td><td class="sorting_1">84</td><td></td><td>Grodno region, Grodno district, highway M-6, 291 km, 1</td><td>+375(152)68-35-18</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=1" width="19px" title="RON-95"><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=18" width="22px" title="hotel"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=230" width="22px" title="Caravan parking"><img src="/beloil-map/geticon.jpg?t=s&amp;id=270" width="22px" title="Adblue in bulk"><img src="/beloil-map/geticon.jpg?t=s&amp;id=350" width="22px" title="Baby changing table"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=7" width="22px" title="E100"><img src="/beloil-map/geticon.jpg?t=p&amp;id=12" width="22px" title="DKV"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=169" width="22px" title="CRT "><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(52.121287, 23.671321, 126);" title="Show on map"></div></td><td class="sorting_1">92</td><td></td><td>Brestoblnefteprodukt, Leitenanta Ryabceva Street, Brest region, Brest, 224025, Belarus</td><td>+375(162)34-06-54</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=1" width="19px" title="RON-95"><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"><img src="/beloil-map/geticon.jpg?t=f&amp;id=128" width="19px" title="RON-98"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=1" width="22px" title="shop"><img src="/beloil-map/geticon.jpg?t=s&amp;id=2" width="22px" title="cafe"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=25" width="22px" title="WC for disabled"><img src="/beloil-map/geticon.jpg?t=s&amp;id=190" width="22px" title="WiFi Free"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=270" width="22px" title="Adblue in bulk"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=2" width="22px" title="Berlio"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=7" width="22px" title="E100"><img src="/beloil-map/geticon.jpg?t=p&amp;id=12" width="22px" title="DKV"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.8964, 30.34055, 147);" title="Show on map"></div></td><td class="sorting_1">99</td><td></td><td>Mogilev region, Mogilev district, Mogilev city, K.Marksa Street, 11</td><td>+375(222)62-82-37</td><td><img src="/beloil-map/geticon.jpg?t=f&amp;id=1" width="19px" title="RON-95"><img src="/beloil-map/geticon.jpg?t=f&amp;id=68" width="19px" title="Diesel"><img src="/beloil-map/geticon.jpg?t=f&amp;id=89" width="19px" title="RON-92"><img src="/beloil-map/geticon.jpg?t=f&amp;id=128" width="19px" title="RON-98"></td><td><img src="/beloil-map/geticon.jpg?t=s&amp;id=10" width="22px" title="Car wash"><img src="/beloil-map/geticon.jpg?t=s&amp;id=11" width="22px" title="compressor"><img src="/beloil-map/geticon.jpg?t=s&amp;id=12" width="22px" title="vacuum cleaner"><img src="/beloil-map/geticon.jpg?t=s&amp;id=210" width="22px" title="Сharging of electric cars"><img src="/beloil-map/geticon.jpg?t=s&amp;id=370" width="22px" title="Full tank"></td><td><img src="/beloil-map/geticon.jpg?t=p&amp;id=1" width="22px" title="Belorusneft (Dallas)"><img src="/beloil-map/geticon.jpg?t=p&amp;id=3" width="22px" title="Bank cards"><img src="/beloil-map/geticon.jpg?t=p&amp;id=148" width="22px" title="Drive&amp;Pay"><img src="/beloil-map/geticon.jpg?t=p&amp;id=189" width="22px" title="Belorusneft (RFID)"></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(52.086524, 23.682281, 18);" title="Show on map"></div></td><td class="sorting_1">Асстор-Вест</td><td></td><td></td><td></td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(55.179468, 30.222975, 130);" title="Show on map"></div></td><td class="sorting_1">г. Витебск, пр. Московский, 60А</td><td></td><td></td><td></td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(55.176868, 30.247535, 131);" title="Show on map"></div></td><td class="sorting_1">г. Витебск, пр. Московский, 86</td><td></td><td></td><td></td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(55.171805, 30.202559, 132);" title="Show on map"></div></td><td class="sorting_1">г. Витебск, пр. Строителей, 1Г</td><td></td><td></td><td></td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(55.187107, 30.190773, 133);" title="Show on map"></div></td><td class="sorting_1">г. Витебск, ул. Димитрова, 31А</td><td></td><td></td><td></td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(55.193956, 30.185935, 134);" title="Show on map"></div></td><td class="sorting_1">г. Витебск, ул. Космонавтов, 10</td><td></td><td></td><td></td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(55.191737, 30.203859, 135);" title="Show on map"></div></td><td class="sorting_1">г. Витебск, ул. Ленина, 26А</td><td></td><td></td><td></td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(55.201829, 30.208518, 136);" title="Show on map"></div></td><td class="sorting_1">г. Витебск, ул. Ленина, 69</td><td></td><td></td><td></td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.931666, 25.770833, 3);" title="Show on map"></div></td><td class="sorting_1">г.Ивье</td><td></td><td>Ivye</td><td></td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(52.092992, 23.667049, 19);" title="Show on map"></div></td><td class="sorting_1">ГОБК,13</td><td></td><td></td><td></td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(52.476785, 30.985387, 2);" title="Show on map"></div></td><td class="sorting_1">Зарядная станция для электромобилей</td><td></td><td></td><td></td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(52.424876, 31.015411, 4);" title="Show on map"></div></td><td class="sorting_1">Зарядная станция для электромобилей площадь Ленина</td><td></td><td></td><td></td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(52.427017, 31.016831, 5);" title="Show on map"></div></td><td class="sorting_1">Зарядная станция для электромобилей по ул.Билецкого</td><td></td><td></td><td></td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(52.423996, 30.99855, 0);" title="Show on map"></div></td><td class="sorting_1">Зарядная станция для электромобилей по ул.Крестьянской</td><td></td><td></td><td></td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(52.086784, 23.667582, 20);" title="Show on map"></div></td><td class="sorting_1">Зубачёва, 25</td><td></td><td></td><td></td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(52.100435, 23.680218, 96);" title="Show on map"></div></td><td class="sorting_1">Кижеватова</td><td></td><td></td><td></td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(52.104554, 23.78789, 29);" title="Show on map"></div></td><td class="sorting_1">Московская, 380А</td><td></td><td>Brestoblnefteprodukt; Moskovskaya street; 224002, Brest region, Brest, Belarus</td><td>+375(29)729-45-01</td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(52.097122, 23.731063, 28);" title="Show on map"></div></td><td class="sorting_1">Пионерская,52</td><td></td><td>Brestoblnefteprodukt; Pionerskaya street; 224002, Brest region, Brest, Belarus</td><td>+375(29)729-45-01</td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(54.517282, 30.146228, 137);" title="Show on map"></div></td><td class="sorting_1">Территория заправочного торгово-производственного комплекса «550 км», а/д М-1 Брест-Минск-граница Российской Федерации,</td><td></td><td></td><td></td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.65053, 23.853833, 23);" title="Show on map"></div></td><td class="sorting_1">ТРЦ «Тринити»</td><td></td><td>Grodno ave. Y. Kupala, 87</td><td></td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.686941, 23.846805, 24);" title="Show on map"></div></td><td class="sorting_1">ЭЗС ЖД-вокзал</td><td></td><td>Grodno Ozheshko st, 42</td><td></td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.710792, 23.815589, 27);" title="Show on map"></div></td><td class="sorting_1">ЭЗС ИООО Корона ул. Горького, 91</td><td></td><td>Grodno Gorky str, 91</td><td></td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.688837, 23.826229, 1);" title="Show on map"></div></td><td class="sorting_1">ЭЗС Моde-3 ул. Коммунальная, 3 Гродно</td><td></td><td>Grodno Communalnaya st, 3</td><td>+375(29)643-14-31</td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(52.461514, 31.039819, 32);" title="Show on map"></div></td><td class="sorting_1">ЭЗС по ул.Чечерской 1Б в г. Гомеле</td><td></td><td></td><td></td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.654255, 23.849679, 26);" title="Show on map"></div></td><td class="sorting_1">ЭЗС ТЦ Евроопт  пр-т Я. Купалы</td><td></td><td>Grodno ave. Y. Kupala, 82-a</td><td></td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.694603, 23.838124, 25);" title="Show on map"></div></td><td class="sorting_1">ЭЗС ТЦ Рублевский</td><td></td><td>Grodno, st. Dzerzhinsky, 54</td><td></td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.68166, 23.836093, 21);" title="Show on map"></div></td><td class="sorting_1">ЭЗС ул. Социалистическая, 14 г. Гродно</td><td></td><td>Grodno city Sotsialisticheskaya st. 14</td><td>+375(29)643-14-31</td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.880783, 30.368014, 58);" title="Show on map"></div></td><td class="sorting_1">ЭЗС №1</td><td></td><td></td><td>+375(232)79-34-37</td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(53.88779, 30.329687, 94);" title="Show on map"></div></td><td class="sorting_1">ЭЗС №2</td><td></td><td></td><td>+375(232)79-34-37</td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(53.901467, 30.341966, 95);" title="Show on map"></div></td><td class="sorting_1">ЭЗС №3</td><td></td><td></td><td>+375(232)79-34-37</td><td></td><td></td><td></td></tr><tr role="row" class="odd"><td><div class="showonmap" onclick="showStationOnMap(52.430034, 30.991732, 30);" title="Show on map"></div></td><td class="sorting_1">ЭЗС, г. Гомель, пл. Привокзальная, 3</td><td></td><td></td><td></td><td></td><td></td><td></td></tr><tr role="row" class="even"><td><div class="showonmap" onclick="showStationOnMap(52.456015, 31.024505, 31);" title="Show on map"></div></td><td class="sorting_1">ЭЗС, г. Гомель, ул.Мазурова, 112</td><td></td><td></td><td></td><td></td><td></td><td></td></tr></tbody>
"""
trees = BeautifulSoup(trs)
trees = trees.find_all("tr")

In [14]:
trees

[<tr class="odd" role="row"><td><div class="showonmap" onclick="showStationOnMap(54.154996, 27.834638, 33);" title="Show on map"></div></td><td class="sorting_1">01</td><td></td><td></td><td></td><td></td><td></td><td></td></tr>,
 <tr class="even" role="row"><td><div class="showonmap" onclick="showStationOnMap(53.931734, 27.511584, 50);" title="Show on map"></div></td><td class="sorting_1">01</td><td></td><td>Minsk, Pobediteley Avenue, 20</td><td>+375(17)279-88-64</td><td></td><td></td><td></td></tr>,
 <tr class="odd" role="row"><td><div class="showonmap" onclick="showStationOnMap(53.846632, 27.655503, 47);" title="Show on map"></div></td><td class="sorting_1">010</td><td></td><td>Minsk, Svislochskaya street, 11</td><td>+375(17)279-88-64</td><td></td><td></td><td></td></tr>,
 <tr class="even" role="row"><td><div class="showonmap" onclick="showStationOnMap(53.899066, 27.565566, 48);" title="Show on map"></div></td><td class="sorting_1">011</td><td></td><td>Minsk, Kirova street, 18</td><

Collating the belarus data into a dataframe and extracting geographical coordinates with the geocode libraries.

In [17]:
belarus = {"number":[],
           "Address":[],
           "lat":[],
           "lng":[]}
for tree in trees:
  txt = tree.get_text('|')
  if "|" in txt:
    txt = txt.split("|")
    if len(txt)>=2:
      num, add = txt[0],txt[1]
      belarus["number"].append(num)
      belarus["Address"].append(add if "+" not in add else "Nil")
  else:
      belarus["number"].append(txt)
      belarus["Address"].append("Nil")
  lat, lng = re.findall(r"\d{1,3}.\d+",str(tree))[:2]
  belarus['lat'].append(float(lat))
  belarus['lng'].append(float(lng))

In [18]:
belarus_df = pd.DataFrame(belarus)
belarus_df.head(5)

,number,Address,lat,lng
0,01,Nil,54.154996,27.834638
1,01,"Minsk, Pobediteley Avenue, 20",53.931734,27.511584
2,010,"Minsk, Svislochskaya street, 11",53.846632,27.655503
3,011,"Minsk, Kirova street, 18",53.899066,27.565566
4,012,"Minsk, Lukyanovicha street, 10",53.944958,27.599032


## Visualization

Reading the required and pre-processed files.

In [19]:
stations_df = pd.read_csv('/content/drive/My Drive/Colab Data/ebvc_routes/stations_df.csv')
belarus_df = pd.read_csv("/content/drive/My Drive/Colab Data/ebvc_routes/belarus_df.csv")

In [20]:
coord_df = pd.read_csv("/content/drive/My Drive/Colab Data/ebvc_routes/PowerPlants_with_Coordinates.csv")

Initializing the map

In [25]:
map_stations = folium.Map(location = [51.5074,-0.1278],
                          tiles='Stamen Terrain',
                          zoom_start=6)
map_stations.add_child(folium.LatLngPopup());

For the next 3 cells we loop through charging stations and apply popups for easy use of map.

In [26]:
for lat, lon, poi, cluster in zip(stations_df['lat'], stations_df['lng'], stations_df['title'], stations_df['address']):
  if -2.5049<lon<38.9905 and 48.0193 < lat < 60.3921:
    label = folium.Popup(f"{poi} \n{cluster}\n({lat},{lon})", parse_html=True)
    folium.Marker( 
        [lat, lon],
        #radius=9,
        popup=label,
         icon=folium.Icon(color= 'blue'),
        ).add_to(map_stations);

In [27]:
for num, address, lat, lon in belarus_df.iloc[1:,:].values:
  pop = f"Number: {num}\n{address}"
  label = folium.Popup(pop, parse_html=True)
  folium.Marker( 
        [lat, lon],
        #radius=9,
        popup=label,
         icon=folium.Icon(color= 'blue'),
        ).add_to(map_stations)
       
map_stations

Output hidden; open in https://colab.research.google.com to view.

In [29]:
for name, site, cap, lat, lon in coord_df[['Name', 'Site', 'Capacity (MW)', 'latitude','longitude']].values:
  if -2.5049<lon<38.9905 and 48.0193 < lat < 60.3921:
    poppin = "{0}\n{1}\n{2}".format(
        name if name != np.nan else "",
        site if site != np.nan else "",
         str(cap) + " MW" if cap != np.nan else "")
    label = folium.Popup(poppin,parse_html=True, max_width="20%")
    folium.Marker( 
        [lat, lon],
        #radius=9,
        popup=label,
         icon=folium.Icon(color='green'),
        ).add_to(map_stations)
        
bp = [51.5014, 0.1419]
rs = [55.7539, 37.6208]
folium.Marker(bp,
              popup = folium.Popup("Buckingham Palace, London, UK."),
              icon= folium.Icon(color='red')
                    ).add_to(map_stations)

folium.Marker(rs,
              popup = folium.Popup("Red Square, Moscow, Russia."),
              icon= folium.Icon(color='red')
                    ).add_to(map_stations)
map_stations

Output hidden; open in https://colab.research.google.com to view.

The map is then saved as an html file

In [ ]:
map_stations.save('/content/drive/My Drive/Colab Data/ebvc_routes/Charging Stations and Speed/Stamen_Terrain_Charging_stations&speed.html')